# import 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data')

In [ ]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map import *

In [ ]:
%load_ext autoreload
%autoreload 2
from stroke_probability_map_1 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import pandas as pd

from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data as Data

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler

from tqdm import tqdm

import pickle

# File and Path

File name:   
EOG_00x_xx_xxx.csv:   
 00x: participant's ID.    
 xx:  index of the recording time.    
 xxx: index of the corresponding eye movements.    
      - e.g., xxx is the index of a training word.   

In [ ]:
result_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Continuous Data/Result'

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous'

In [ ]:
result_path_isolated = '/content/drive/MyDrive/Colab Notebooks/MLMA/project/Code Isolated Data/Result'

# Data explore

In [ ]:
len(file_list)

756

In [ ]:
frequency = np.zeros((6, 150))
length = []
for ID in range(1, 7):
  c = 0
  p = data_path + '/00' + str(ID) +'/training/'
  file_list = os.listdir(p)
  length.append([])
  for i in range(len(file_list)):
    if '.csv' in file_list[i]:
      a = pd.read_csv(p + file_list[i])
      length[-1].append(a.shape[0])
      frequency[ID-1, int(file_list[i][-7:-4])-1] += 1
      c+=1

In [ ]:
t,f = np.unique(frequency, return_counts=True)

In [ ]:
print(t)
print(f)

[3. 4. 5. 6. 7. 8.]
[  1   1 868  23   5   2]


In [ ]:
for i in range(6):
  print(np.min(length[i]), np.max(length[i]), np.mean(length[i]))

224 599 422.72266666666667
136 486 279.0733333333333
149 536 317.271164021164
211 836 439.542216358839
274 1136 607.5059132720105
161 761 403.9658344283837


# Dataset and Dataloader

In [ ]:
class MyDataset(Dataset):
  def __init__(self, ID_list, R_time_list, model_list, flatten, model_type, root_dir='/content/drive/MyDrive/Colab Notebooks/MLMA/project/Data/continuous/00'):
    self.data = []
    self.stroke_probability_map = []
    self.label = np.zeros((0,))
    for id_idx in range(len(ID_list)):
      file_list = os.listdir(root_dir + str(ID_list[id_idx]) +'/training/')
      for f_idx in range(len(file_list)):
        if int(file_list[f_idx][8:10]) in R_time_list:
          data = pd.read_csv(root_dir + str(ID_list[id_idx]) +'/training/' + file_list[f_idx],  header=None).to_numpy()
          #print(data.shape)
          data = signal.resample(data, 1024, axis=0)
          #print(data.shape)
          s = stroke_probability_map_1(data, model_list[id_idx], flatten, model_type)
          s = torch.from_numpy(s)
          self.stroke_probability_map.append(s)
          self.label = np.append(self.label, np.ones((1,))*(int(file_list[f_idx][-7:-4])-1), axis=0)
          data = (data - data.mean(axis=0, keepdims=True))/np.std(data,axis=0, keepdims=True)
          data = torch.from_numpy(data)
          self.data.append(data)

    self.label = torch.from_numpy(self.label)
  def __getitem__(self, index) :
    return self.data[index].float(), self.stroke_probability_map[index].float(), self.label[index].type(torch.LongTensor)
  def __len__(self):
    return len(self.data)

# Model

https://www.cnblogs.com/marsggbo/p/8572846.html

SPP

In [ ]:
# 一般
class My_CNN_1_simplify2(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_1_simplify2, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=9, padding=5)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, padding=3)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=3, padding=2)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, padding=2)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(3*12+16), 25)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)


  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
# Overfitting 好一些
class My_CNN_1_simplify(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_1_simplify, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=9, padding=5)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=5, padding=3)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=2)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(3*12+32), 25)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)


  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
# 比3复杂
class My_CNN_3(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_3, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(3*12+256), 25)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
# 2+lstm
class My_CNN_2_lstm(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_2_lstm, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()

    self.lstm = nn.LSTM(input_size=int(3*12+128), hidden_size=25, num_layers=1, batch_first=True, bidirectional=False, dropout=0.5)
    for name, param in self.lstm.named_parameters():
      if name.startswith("weight"):
        nn.init.xavier_normal_(param)
      else:
        nn.init.zeros_(param)

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x, (_, _) = self.lstm(x)
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
# 比1复杂
class My_CNN_2(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_2, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=5)
    nn.init.xavier_uniform_(self.conv1.weight)
    nn.init.constant_(self.conv1.bias, 0.0)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=3)
    nn.init.xavier_uniform_(self.conv2.weight)
    nn.init.constant_(self.conv2.bias, 0.0)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv3.weight)
    nn.init.constant_(self.conv3.bias, 0.0)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
    nn.init.xavier_uniform_(self.conv4.weight)
    nn.init.constant_(self.conv4.bias, 0.0)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(3*12+128), 25)
    nn.init.xavier_uniform_(self.fc1.weight)
    nn.init.constant_(self.fc1.bias, 0.0)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)
    nn.init.xavier_uniform_(self.fc2.weight)
    nn.init.constant_(self.fc2.bias, 0.0)

  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
# Train 收敛效果好，但是overfitting严重
class My_CNN_1(nn.Module):
  def __init__(self, N=16):
    super(My_CNN_1, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=5)
    self.mp1 = nn.MaxPool1d(kernel_size=4)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=3)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    

    self.fc1 = nn.Linear(int(3*12+32), 25)
    self.relu5 = nn.LeakyReLU()

    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(p=0.5)

    self.fc2 = nn.Linear(25*16, 150)


  def forward(self, x, probability):
    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = torch.transpose(x, -1, -2)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

### x

In [ ]:
class My_CNN_2(nn.Module):
  def __init__(self, N=10):
    super(My_CNN_2, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=2)
    #self.mp1 = nn.MaxPool1d(kernel_size=2)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
    #self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)
    #self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=32, out_channels=36, kernel_size=3, padding=2)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    
    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(int(45*12+N*36), 512)
    self.relu5 = nn.LeakyReLU()

    self.fc2 = nn.Linear(512, 150)


  def forward(self, x, probability):
    probability = self.flatten(probability)

    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.conv1(x))
    x = self.relu2(self.conv2(x))
    x = self.relu3(self.conv3(x))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = self.flatten(x)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.fc2(x)

    return x

In [ ]:
class My_CNN_original(nn.Module):
  def __init__(self, N=10):
    super(My_CNN_original, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=2)
    self.mp1 = nn.MaxPool1d(kernel_size=2)
    self.relu1 = nn.LeakyReLU()

    self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
    self.mp2 = nn.MaxPool1d(kernel_size=2)
    self.relu2 = nn.LeakyReLU()

    self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)
    self.mp3 = nn.MaxPool1d(kernel_size=2)
    self.relu3 = nn.LeakyReLU()

    self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)
    self.amp =  nn.AdaptiveMaxPool1d(N)
    self.sigmoid = nn.Sigmoid()
    
    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(int(45*12+N*32), 512)
    self.relu5 = nn.LeakyReLU()

    self.fc2 = nn.Linear(512, 150)


  def forward(self, x, probability):
    probability = self.flatten(probability)

    x = torch.transpose(x, -1, -2)

    x = self.relu1(self.mp1(self.conv1(x)))
    x = self.relu2(self.mp2(self.conv2(x)))
    x = self.relu3(self.mp3(self.conv3(x)))
    x = self.conv4(x)
    x = self.sigmoid(self.amp(x))
    x = self.flatten(x)
    x = torch.cat((x, probability), dim=-1)

    x = self.relu5(self.fc1(x))
    x = self.fc2(x)

    return x

# Some Funtions

In [ ]:
def Train_model(model,epochs,optmizer,criterion, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    train_loss = []
    x = 0
    for batch_idx, (data, probability, label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        probability = probability.to(device)
        optmizer.zero_grad()  
        output = model(data, probability)
        # print(type(output), type(label))
        loss = criterion(output, label)
        loss.backward()
        optmizer.step() 
        train_loss.append(loss.item()) 
        #if(batch_idx+1)%3 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.\
                #format(epochs, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(train_loss), model

In [ ]:
def test(model,epochs,criterion, dataloader, T_T='Test'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    c = 0
    y_pred = np.zeros((0,))
    y_true = np.zeros((0, ))
    with torch.no_grad():
        for data, probability, label in dataloader:
            data = data.to(device)
            label = label.to(device)
            probability = probability.to(device)
            output = model(data, probability)
            test_loss += criterion(output, label).item()
            y_pred = np.append(y_pred, output.cpu().argmax(dim=-1), axis=0)
            y_true = np.append(y_true, label.cpu(), axis=0)
            c += 1
    test_loss /= c
    #if epochs%50 == 0:
        #print('EPOCH:{}    {} set: Average loss: {:.4f}'.format(epochs, T_T, test_loss))
  
    return test_loss, accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average='micro', zero_division=0), \
    f1_score(y_true, y_pred, average='macro', zero_division=0), precision_score(y_true, y_pred, average='micro', zero_division=0), \
    precision_score(y_true, y_pred, average='macro', zero_division=0), recall_score(y_true, y_pred, average='micro', zero_division=0), \
    recall_score(y_true, y_pred, average='macro', zero_division=0)

In [ ]:
def main_function(total_epoch=100, dataloader_list=None, model_para=None, model_save=False, model_name=None, cross_val=True, weight_decay=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = My_CNN_2_lstm().to(device)
        
    optmizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)  
    #optmizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=weight_decay) 
    scheduler = lr_scheduler.StepLR(optmizer, step_size=50, gamma=0.5, verbose=False) 
    criterion = nn.CrossEntropyLoss().to(device)  
    
    train_loss = np.zeros((total_epoch))
    score = np.zeros((len(dataloader_list), 8, total_epoch)) # loss, accuracy, f1 score*2, precision*2, recall*2
    
    for i in tqdm(range(total_epoch)):
        train_loss[i], model = Train_model(model, i, optmizer, criterion, dataloader_list[0])
        scheduler.step()
        for j in range(len(dataloader_list)):
            if j == 0:
                T_T = 'Train'
            elif j == 1:
                T_T = 'Validation'
            else:
                T_T = 'Test'
            score[j, :, i] = test(model, i, criterion, dataloader_list[j], T_T=T_T)
        #if np.argmin(score[1, 0, :(i+1)]) == i:
            #if model_save:
                #torch.save(model.state_dict(), model_name)     
    #if model_save: 
        #np.savez(model_name[:-4], train_loss=train_loss, score=score)
    if model_save:
      torch.save(model.state_dict(), model_name)
    if cross_val:
        epoch_best = np.argmin(score[1, 0, :])
        return score[:, :, epoch_best]
    else:
        return train_loss, score

 # User Dependent

In [ ]:
score = np.zeros((6, 5, 3, 8))
BATCH_SIZE = 128
for ID in range(1, 7):
  model_name = result_path_isolated + '/mlp_sub00' + str(ID) + '.pck'
  model_stroke = pickle.load(open(model_name, "rb"))

  for fold_idx in range(1, 6):
    test_dataset = MyDataset(ID_list=[ID], R_time_list=[fold_idx], model_list=[model_stroke], flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list=[ID], R_time_list=[x], model_list=[model_stroke], flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list=[ID], R_time_list=R_time_list, model_list=[model_stroke], flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


    model_param=64
    score[ID-1, fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                            model_para=model_param, model_save=False, cross_val=True)
    print('ID{} fold{} Train Loss{}, Train Acc{}'.format(ID, fold_idx, score[ID-1, fold_idx-1, 0, 0], score[ID-1, fold_idx-1, 0, 1]))
    print('ID{} fold{} Val Loss{}, Val Acc{}'.format(ID, fold_idx, score[ID-1, fold_idx-1, 1, 0], score[ID-1, fold_idx-1, 1, 1]))
    print('ID{} fold{} Test Loss{}, Test Acc{}'.format(ID, fold_idx, score[ID-1, fold_idx-1, 2, 0], score[ID-1, fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()
    break
  break

# User Dependent 2

## ***d0419***

## Leave one trial out

In [ ]:
score = np.zeros((4, 3, 8))
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_list = []
for ID in range(1, 7):
  model_name = result_path_isolated + '/SVC_sub00' + str(ID) + '.pck'
  model_stroke = pickle.load(open(model_name, "rb"))
  model_list.append(model_stroke)


for fold_idx in range(1, 5):
  test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
  test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

  x = fold_idx+1 if fold_idx+1<=5 else 1
  val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
  val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

  R_time_list = []
  for i in range(1, 9):
    if i!=fold_idx and i!=x:
      R_time_list.append(i)
  train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
  train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


  model_param=64
  score[fold_idx-1] = main_function(total_epoch=500, dataloader_list=[train_loader, val_loader, test_loader], \
                              model_para=model_param, model_save=False, cross_val=True)
  print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
  print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
  print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
  torch.cuda.empty_cache()

np.save(result_path + '/d0419_score_7', score)

**weight_decay = 1e-3**



MLP

In [ ]:
# My_CNN_1
x = np.mean(score, axis=0)
print('Train', x[0])
print('Val', x[1])
print('Test', x[2])

Train [0.02044575 1.         1.         1.         1.         1.
 1.         1.        ]
Val [0.55072433 0.84569883 0.84569883 0.84499433 0.84569883 0.86426112
 0.84569883 0.84566667]
Test [0.54708268 0.84857156 0.84857156 0.84705738 0.84857156 0.86606223
 0.84857156 0.84855556]


In [ ]:
# My_CNN_1_simplify
x = np.mean(score, axis=0)
print('Train', x[0])
print('Val', x[1])
print('Test', x[2])

Train [0.02857823 1.         1.         1.         1.         1.
 1.         1.        ]
Val [0.72329674 0.80039631 0.80039631 0.79769884 0.80039631 0.81597817
 0.80039631 0.80044444]
Test [0.71039559 0.80328606 0.80328606 0.80174082 0.80328606 0.82290873
 0.80328606 0.80327778]


In [ ]:
# My_CNN_2
x = np.mean(score, axis=0)
print('Train', x[0])
print('Val', x[1])
print('Test', x[2])

Train [0.003943 1.       1.       1.       1.       1.       1.       1.      ]
Val [0.26003276 0.92354711 0.92354711 0.92340685 0.92354711 0.93439322
 0.92354711 0.9235    ]
Test [0.25467589 0.92747961 0.92747961 0.92707469 0.92747961 0.93722421
 0.92747961 0.92744444]


SVC

In [ ]:
# My_CNN_2
x = np.mean(score, axis=0)
print('Train', x[0])
print('Val', x[1])
print('Test', x[2])

Train [1.25780054 0.75164294 0.75164294 0.75002176 0.75164294 0.75001095
 0.75164294 0.75166667]
Val [1.53292642 0.67551848 0.67551848 0.67342644 0.67551848 0.6840946
 0.67551848 0.67583333]
Test [1.54590309 0.67333519 0.67333519 0.67133243 0.67333519 0.6803438
 0.67333519 0.67333333]


**weight_decay 2e-3, 5e-3**

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
weight_decay_list = [2e-3, 5e-3]

model_name_list = ['mlp', 'SVC']
save_number = 8
for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  for w_idx in range(2):
    score = np.zeros((4, 3, 8))
    for fold_idx in range(1, 5):
      test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
      test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

      x = fold_idx+1 if fold_idx+1<=5 else 1
      val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
      val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

      R_time_list = []
      for i in range(1, 9):
        if i!=fold_idx and i!=x:
          R_time_list.append(i)
      train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
      train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
      model_param=64
      score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, \
                                  weight_decay=weight_decay_list[w_idx])
      print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
      print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
      print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
      torch.cuda.empty_cache()

    np.save(result_path + '/d0419_score_' + str(save_number), score)
    save_number += 1

    x = np.mean(score, axis=0)
    print('Train', x[0])
    print('Val', x[1])
    print('Test', x[2])

In [ ]:
l = ['MLP 2e-3', 'MLP 5e-3', 'SVC 2e-3', 'SVC 5e-3']
for i in range(8, 12):
  score = np.load(result_path + '/d0419_score_' + str(i) + '.npy')
  x = np.mean(score, axis=0)
  print(l[i-8])
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

MLP 2e-3
Train [0.01091593 1.         1.         1.         1.         1.
 1.         1.        ]
Val [0.28157514 0.92577087 0.92577087 0.9255994  0.92577087 0.93565278
 0.92577087 0.92583333]
Test [0.29266739 0.91914504 0.91914504 0.91789624 0.91914504 0.92898214
 0.91914504 0.91916667]
MLP 5e-3
Train [0.04894792 0.99972598 0.99972598 0.99973826 0.99972598 0.99974123
 0.99972598 0.99974868]
Val [0.40448756 0.89046094 0.89046094 0.88947508 0.89046094 0.90389881
 0.89046094 0.89044444]
Test [0.38996305 0.90164226 0.90164226 0.90106373 0.90164226 0.9141045
 0.90164226 0.90161111]
SVC 2e-3
Train [3.76142782 0.25511632 0.25511632 0.25017655 0.25511632 0.25009078
 0.25511632 0.25505361]
Val [3.84923661 0.22577054 0.22577054 0.22120742 0.22577054 0.22540468
 0.22577054 0.22577778]
Test [3.84890309 0.22997126 0.22997126 0.22345567 0.22997126 0.22706479
 0.22997126 0.23      ]
SVC 5e-3
Train [5.01068534e+00 6.39413007e-03 6.39413007e-03 5.26243535e-04
 6.39413007e-03 1.06877160e-03 6.39413007e

**My_CNN_3**

MLP/SVC

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 12

for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True)
    print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
    print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
    print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0419_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 300/300 [02:00<00:00,  2.49it/s]


fold1 Train Loss0.002566661294126375, Train Acc1.0
fold1 Val Loss0.28227162521216087, Val Acc0.9233333333333333
fold1 Test Loss0.25943817313964246, Test Acc0.9255555555555556


100%|██████████| 300/300 [02:02<00:00,  2.45it/s]


fold2 Train Loss0.003434820746240968, Train Acc1.0
fold2 Val Loss0.3122419146820903, Val Acc0.9211111111111111
fold2 Test Loss0.29303138982504606, Test Acc0.91


100%|██████████| 300/300 [02:00<00:00,  2.48it/s]


...

fold2 Train Loss5.010097243569114, Train Acc0.004751461988304093
fold2 Val Loss5.009352624416351, Val Acc0.006666666666666667
fold2 Test Loss5.00787490606308, Test Acc0.01


100%|██████████| 300/300 [01:59<00:00,  2.51it/s]


fold3 Train Loss5.010604338212446, Train Acc0.006576543660942638
fold3 Val Loss5.009983003139496, Val Acc0.006674082313681869
fold3 Test Loss5.010462462902069, Test Acc0.006666666666666667


100%|██████████| 300/300 [02:00<00:00,  2.49it/s]

fold4 Train Loss5.0103866837241435, Train Acc0.007667031763417305
fold4 Val Loss5.011408805847168, Val Acc0.0066815144766146995
fold4 Test Loss5.011310040950775, Test Acc0.006674082313681869
Train [5.01040642e+00 6.66761900e-03 6.66761900e-03 5.64062396e-04
 6.66761900e-03 1.95903890e-03 6.66761900e-03 6.15824283e-03]
Val [5.01029958e+00 6.67223253e-03 6.67223253e-03 2.39752393e-04
 6.67223253e-03 1.27301096e-04 6.67223253e-03 6.66666667e-03]
Test [5.01020400e+00 7.50185391e-03 7.50185391e-03 5.53670361e-04
 7.50185391e-03 3.95435673e-04 7.50185391e-03 7.50000000e-03]


In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 14

for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=2e-3)
    print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
    print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
    print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0419_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 300/300 [01:59<00:00,  2.50it/s]


fold1 Train Loss5.009634082967585, Train Acc0.01023391812865497
fold1 Val Loss5.010189235210419, Val Acc0.005555555555555556
fold1 Test Loss5.010197579860687, Test Acc0.006666666666666667


100%|██████████| 300/300 [02:03<00:00,  2.44it/s]


fold2 Train Loss4.320016557520086, Train Acc0.14839181286549707
fold2 Val Loss4.552741646766663, Val Acc0.07888888888888888
fold2 Test Loss4.483769714832306, Test Acc0.09444444444444444


100%|██████████| 300/300 [02:03<00:00,  2.44it/s]


...

fold2 Train Loss5.011329564181241, Train Acc0.006578947368421052
fold2 Val Loss5.0086387395858765, Val Acc0.006666666666666667
fold2 Test Loss5.012802600860596, Test Acc0.006666666666666667


100%|██████████| 300/300 [02:00<00:00,  2.48it/s]


fold3 Train Loss5.010767438194969, Train Acc0.006576543660942638
fold3 Val Loss5.009313523769379, Val Acc0.006674082313681869
fold3 Test Loss5.010397374629974, Test Acc0.006666666666666667


100%|██████████| 300/300 [02:01<00:00,  2.47it/s]

fold4 Train Loss5.0111802057786425, Train Acc0.007667031763417305
fold4 Val Loss5.009488344192505, Val Acc0.0066815144766146995
fold4 Test Loss5.022032678127289, Test Acc0.006674082313681869
Train [5.01098650e+00 6.66761900e-03 6.66761900e-03 9.29730824e-05
 6.66761900e-03 4.68309289e-05 6.66761900e-03 6.40350877e-03]
Val [5.00948758e+00 5.83889920e-03 5.83889920e-03 7.97936732e-05
 5.83889920e-03 4.01731602e-05 5.83889920e-03 5.83333333e-03]
Test [5.01388985e+00 6.11296502e-03 6.11296502e-03 8.44396443e-05
 6.11296502e-03 4.25157982e-05 6.11296502e-03 6.11111111e-03]


In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 16

for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
    print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
    print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0419_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


fold1 Train Loss0.0005534725866957822, Train Acc1.0
fold1 Val Loss0.2497770295594819, Val Acc0.9288888888888889
fold1 Test Loss0.2479567216223586, Test Acc0.9277777777777778


100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


fold2 Train Loss0.0005557983968174085, Train Acc1.0
fold2 Val Loss0.24993076687678695, Val Acc0.9388888888888889
fold2 Test Loss0.2433132279838901, Test Acc0.9333333333333333


100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


...

fold2 Train Loss0.0003199148208791898, Train Acc1.0
fold2 Val Loss0.28129118151264265, Val Acc0.93
fold2 Test Loss0.24922210826480296, Test Acc0.9288888888888889


100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


fold3 Train Loss0.00029737329499436203, Train Acc1.0
fold3 Val Loss0.24535674609069247, Val Acc0.9343715239154616
fold3 Test Loss0.25852770069286635, Test Acc0.9433333333333334


100%|██████████| 300/300 [02:04<00:00,  2.41it/s]

fold4 Train Loss0.00018733346157453278, Train Acc1.0
fold4 Val Loss0.20672691779100205, Val Acc0.9409799554565702
fold4 Test Loss0.22770134952588705, Test Acc0.9454949944382648
Train [4.42187108e-04 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
Val [0.2492905  0.93272676 0.93272676 0.93202133 0.93272676 0.94087302
 0.93272676 0.93272222]
Test [0.24891206 0.93581819 0.93581819 0.93532649 0.93581819 0.94445274
 0.93581819 0.93583333]


In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 18

for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
    print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
    print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0419_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


fold1 Train Loss0.0005034695244500075, Train Acc1.0
fold1 Val Loss0.2613779172716022, Val Acc0.9344444444444444
fold1 Test Loss0.24747333677805727, Test Acc0.9255555555555556


100%|██████████| 300/300 [02:03<00:00,  2.43it/s]


fold2 Train Loss0.00042458974878007376, Train Acc1.0
fold2 Val Loss0.26338601240422577, Val Acc0.9366666666666666
fold2 Test Loss0.22460568789028912, Test Acc0.9377777777777778


100%|██████████| 300/300 [02:05<00:00,  2.38it/s]


...

fold2 Train Loss0.0005160212762844326, Train Acc1.0
fold2 Val Loss0.2967454297468066, Val Acc0.93
fold2 Test Loss0.24449375856193, Test Acc0.9344444444444444


100%|██████████| 300/300 [02:04<00:00,  2.40it/s]


fold3 Train Loss0.0005181974636136808, Train Acc1.0
fold3 Val Loss0.24342807995708426, Val Acc0.9310344827586207
fold3 Test Loss0.3274388900026679, Test Acc0.9277777777777778


100%|██████████| 300/300 [02:05<00:00,  2.40it/s]

fold4 Train Loss0.0004921906035054814, Train Acc1.0
fold4 Val Loss0.23523471796397644, Val Acc0.9320712694877505
fold4 Test Loss0.25342347437981516, Test Acc0.9310344827586207
Train [5.08048013e-04 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
Val [0.27084706 0.92938755 0.92938755 0.92880427 0.92938755 0.93819048
 0.92938755 0.92938889]
Test [0.26197119 0.93220307 0.93220307 0.93196306 0.93220307 0.94118056
 0.93220307 0.93216667]


## ***d0420***

## Leave one subject out

In [ ]:
BATCH_SIZE = 128
R_time_list = [1,2,3,4,5,6, 7, 8]
model_name_list = ['mlp', 'SVC']
save_number = 0

for m_idx in range(2):
  # Load model
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  # Leave subject out
  score = np.zeros((6, 3, 8))
  for ID in range(1, 7):
    test_dataset = MyDataset([ID], R_time_list=R_time_list, model_list=[model_list[ID-1]], flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = ID+1 if ID+1<=6 else 1
    val_dataset = MyDataset([x], R_time_list=R_time_list, model_list=[model_list[x-1]], flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    Model_list_train = []
    ID_list_train = []
    for i in range(1, 7):
      if i!=ID and i!=x:
        Model_list_train.append(model_list[i-1])
        ID_list_train.append(i)
    train_dataset = MyDataset(ID_list_train, R_time_list=R_time_list, model_list=Model_list_train, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[ID-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('ID{} Train Loss{}, Train Acc{}'.format(ID, score[ID-1, 0, 0], score[ID-1, 0, 1]))
    print('ID{} Val Loss{}, Val Acc{}'.format(ID, score[ID-1, 1, 0], score[ID-1, 1, 1]))
    print('ID{} Test Loss{}, Test Acc{}'.format(ID, score[ID-1, 2, 0], score[ID-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

  0%|          | 1/300 [00:08<40:11,  8.06s/it]

EPOCH:0    Train set: Average loss: 5.0077
EPOCH:0    Validation set: Average loss: 5.0103
EPOCH:0    Test set: Average loss: 5.0119


 17%|█▋        | 51/300 [00:27<01:35,  2.61it/s]

EPOCH:50    Train set: Average loss: 0.0023
EPOCH:50    Validation set: Average loss: 3.0914
EPOCH:50    Test set: Average loss: 5.6473


 34%|███▎      | 101/300 [00:46<01:16,  2.62it/s]

...

EPOCH:200    Train set: Average loss: 0.0004
EPOCH:200    Validation set: Average loss: 4.5393
EPOCH:200    Test set: Average loss: 0.7107


 84%|████████▎ | 251/300 [01:41<00:19,  2.46it/s]

EPOCH:250    Train set: Average loss: 0.0004
EPOCH:250    Validation set: Average loss: 4.3788
EPOCH:250    Test set: Average loss: 0.6867


100%|██████████| 300/300 [02:01<00:00,  2.48it/s]

ID6 Train Loss0.07251232365767162, Train Acc0.9937190082644628
ID6 Val Loss3.8100925286610923, Val Acc0.288
ID6 Test Loss0.8774158159891764, Test Acc0.7634691195795007
Train [0.06143511 0.99235342 0.99235342 0.99236854 0.99235342 0.99268854
 0.99235342 0.99239392]
Val [2.63827035 0.49158245 0.49158245 0.46025467 0.49158245 0.50088573
 0.49158245 0.49302513]
Test [2.95941103 0.46695702 0.46695702 0.43629489 0.46695702 0.47966021
 0.46695702 0.46726984]


In [ ]:
BATCH_SIZE = 512
R_time_list = [1,2,3,4,5,6, 7, 8]
model_name_list = ['mlp', 'SVC']
save_number = 2

for m_idx in range(2):
  # Load model
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  # Leave subject out
  score = np.zeros((6, 3, 8))
  for ID in range(1, 7):
    test_dataset = MyDataset([ID], R_time_list=R_time_list, model_list=[model_list[ID-1]], flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = ID+1 if ID+1<=6 else 1
    val_dataset = MyDataset([x], R_time_list=R_time_list, model_list=[model_list[x-1]], flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    Model_list_train = []
    ID_list_train = []
    for i in range(1, 7):
      if i!=ID and i!=x:
        Model_list_train.append(model_list[i-1])
        ID_list_train.append(i)
    train_dataset = MyDataset(ID_list_train, R_time_list=R_time_list, model_list=Model_list_train, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[ID-1] = main_function(total_epoch=200, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=2e-3)
    print('ID{} Train Loss{}, Train Acc{}'.format(ID, score[ID-1, 0, 0], score[ID-1, 0, 1]))
    print('ID{} Val Loss{}, Val Acc{}'.format(ID, score[ID-1, 1, 0], score[ID-1, 1, 1]))
    print('ID{} Test Loss{}, Test Acc{}'.format(ID, score[ID-1, 2, 0], score[ID-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

100%|██████████| 200/200 [01:04<00:00,  3.12it/s]


ID1 Train Loss0.01182405836880207, Train Acc1.0
ID1 Val Loss2.618264675140381, Val Acc0.504
ID1 Test Loss4.607635736465454, Test Acc0.192


100%|██████████| 200/200 [01:04<00:00,  3.11it/s]


ID2 Train Loss0.013154251345743736, Train Acc1.0
ID2 Val Loss2.1136680841445923, Val Acc0.5423280423280423
ID2 Test Loss1.8561499118804932, Test Acc0.552


100%|██████████| 200/200 [01:04<00:00,  3.12it/s]


...

ID4 Train Loss0.02242873205492894, Train Acc1.0
ID4 Val Loss1.7180609107017517, Val Acc0.6478318002628121
ID4 Test Loss4.1731343269348145, Test Acc0.2955145118733509


100%|██████████| 200/200 [01:04<00:00,  3.12it/s]


ID5 Train Loss0.03173454447338978, Train Acc0.9993364299933643
ID5 Val Loss1.9947250485420227, Val Acc0.5663600525624178
ID5 Test Loss2.5776039361953735, Test Acc0.5045992115637319


100%|██████████| 200/200 [01:03<00:00,  3.16it/s]

ID6 Train Loss0.033676002795497574, Train Acc0.9996694214876033
ID6 Val Loss4.305452108383179, Val Acc0.19866666666666666
ID6 Test Loss0.7733030617237091, Test Acc0.783180026281209
Train [0.59994637 0.8808739  0.8808739  0.8758912  0.8808739  0.88285743
 0.8808739  0.88096846]
Val [3.15515043 0.38992713 0.38992713 0.36169724 0.38992713 0.39856902
 0.38992713 0.3910463 ]
Test [3.3881712  0.37482056 0.37482056 0.34902759 0.37482056 0.38410254
 0.37482056 0.37496429]


## Leave one trial out LSTM

In [ ]:
BATCH_SIZE = 128
ID_list = [1,2,3,4,5,6]
model_name_list = ['mlp', 'SVC']
save_number = 4

for m_idx in range(2):
  model_list = []
  for ID in range(1, 7):
    model_name = result_path_isolated + '/'+ model_name_list[m_idx] + '_sub00' + str(ID) + '.pck'
    model_stroke = pickle.load(open(model_name, "rb"))
    model_list.append(model_stroke)

  score = np.zeros((4, 3, 8))
  for fold_idx in range(1, 5):
    test_dataset = MyDataset(ID_list, R_time_list=[fold_idx], model_list=model_list, flatten=True, model_type=0)
    test_loader = Data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    x = fold_idx+1 if fold_idx+1<=5 else 1
    val_dataset = MyDataset(ID_list, R_time_list=[x], model_list=model_list, flatten=True, model_type=0)
    val_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    R_time_list = []
    for i in range(1, 9):
      if i!=fold_idx and i!=x:
        R_time_list.append(i)
    train_dataset = MyDataset(ID_list, R_time_list=R_time_list, model_list=model_list, flatten=True, model_type=0)
    train_loader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

      
    model_param=64
    score[fold_idx-1] = main_function(total_epoch=300, dataloader_list=[train_loader, val_loader, test_loader], \
                                  model_para=model_param, model_save=False, cross_val=True, weight_decay=1e-4)
    print('fold{} Train Loss{}, Train Acc{}'.format(fold_idx, score[fold_idx-1, 0, 0], score[fold_idx-1, 0, 1]))
    print('fold{} Val Loss{}, Val Acc{}'.format(fold_idx, score[fold_idx-1, 1, 0], score[fold_idx-1, 1, 1]))
    print('fold{} Test Loss{}, Test Acc{}'.format(fold_idx, score[fold_idx-1, 2, 0], score[fold_idx-1, 2, 1]))
    torch.cuda.empty_cache()

  np.save(result_path + '/d0420_score_' + str(save_number), score)
  save_number += 1

  x = np.mean(score, axis=0)
  print('Train', x[0])
  print('Val', x[1])
  print('Test', x[2])

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 300/300 [01:57<00:00,  2.54it/s]


fold1 Train Loss0.00413815321570093, Train Acc1.0
fold1 Val Loss0.15441490150988102, Val Acc0.9566666666666667
fold1 Test Loss0.14860972340102307, Test Acc0.95


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 300/300 [02:00<00:00,  2.48it/s]


fold2 Train Loss0.0036076848098838873, Train Acc1.0
fold2 Val Loss0.15129810012876987, Val Acc0.96
fold2 Test Loss0.14222064922796562, Test Acc0.9555555555555556


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 300/300 [02:00<00:00,  2.49it/s]


...

fold2 Train Loss0.004541903649541465, Train Acc1.0
fold2 Val Loss0.20734457485377789, Val Acc0.9477777777777778
fold2 Test Loss0.15044284294708632, Test Acc0.9511111111111111


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 300/300 [02:02<00:00,  2.44it/s]


fold3 Train Loss0.005345547635277564, Train Acc1.0
fold3 Val Loss0.1699959586840123, Val Acc0.9377085650723026
fold3 Test Loss0.1969612711109221, Test Acc0.95


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 300/300 [02:04<00:00,  2.42it/s]

fold4 Train Loss0.004829184850677848, Train Acc1.0
fold4 Val Loss0.13902956241508946, Val Acc0.9543429844097996
fold4 Test Loss0.16824372136034071, Test Acc0.9454949944382648
Train [0.00493578 1.         1.         1.         1.         1.
 1.         1.        ]
Val [0.166471   0.95023511 0.95023511 0.9498838  0.95023511 0.95663294
 0.95023511 0.95022222]
Test [0.16122529 0.95081819 0.95081819 0.95066475 0.95081819 0.95740079
 0.95081819 0.95083333]
